<a href="https://colab.research.google.com/github/Latticeworks1/SEE-PT-AI/blob/main/SBT_PREDICTION_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from google.colab import files

# Define a class to encapsulate the SBT model functionality
class SBTModel:
    def __init__(self):
        self.model = None
        self.scaler = None

    def train(self, X, y, num_layers, num_neurons):
        # Normalize the input data
        self.scaler = MinMaxScaler()
        X = self.scaler.fit_transform(X)

        # Create a neural network model
        self.model = Sequential()
        self.model.add(Dense(num_neurons, activation='relu', input_dim=X.shape[1]))
        self.model.add(Dropout(0.2))

        for _ in range(num_layers-1):
            self.model.add(Dense(num_neurons, activation='relu'))
            self.model.add(Dropout(0.2))

        self.model.add(Dense(1))

        # Compile the model
        optimizer = Adam(learning_rate=0.001)
        self.model.compile(optimizer=optimizer, loss='mean_squared_error')

        # Train the model
        self.model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2)

    def predict(self, X):
        # Normalize the input data using the same scaler
        X = self.scaler.transform(X)

        # Make predictions using the trained model
        predictions = self.model.predict(X)
        return predictions.flatten()


# Create the SBT model object
sbt_model = SBTModel()

# Function to handle the 'Train' button click event
def train_model(button):
    # Upload the training data file
    uploaded_train = files.upload()
    train_data = pd.read_csv(next(iter(uploaded_train)), delimiter=r'\s+|[\t,]', engine='python', skiprows=1, header=None)
    X_train = train_data.iloc[:, :5].values  # Use the first 5 columns as input features
    y_train = train_data.iloc[:, 5].values  # SBT as the target variable

    # Get the selected values from the sliders
    num_layers = num_layers_slider.value
    num_neurons = num_neurons_slider.value

    # Train the SBT model
    sbt_model.train(X_train, y_train, num_layers, num_neurons)

    print("Training completed successfully.")


# Create sliders for adjusting the hyperparameters
num_layers_slider = widgets.IntSlider(value=2, min=1, max=5, description='Num Layers')
num_neurons_slider = widgets.IntSlider(value=64, min=16, max=256, step=16, description='Num Neurons')

# Create a button for training the model
train_button = widgets.Button(description='Train')
train_button.on_click(train_model)

# Display the GUI elements
display(num_layers_slider)
display(num_neurons_slider)
display(train_button)


# Function to handle the 'Predict' button click event
def predict_model(button):
    # Upload the new data file for prediction
    uploaded_new = files.upload()
    new_data = pd.read_csv(next(iter(uploaded_new)), delimiter=r'\s+|[\t,]', engine='python', skiprows=1, header=None)
    X_new = new_data.iloc[:, :5].values  # Use the first 5 columns as input features

    # Make predictions using the trained model
    predictions = sbt_model.predict(X_new)

    # Round the predicted SBT values
    rounded_predictions = np.round(predictions)

    # Print the rounded predicted SBT values
    print("Rounded Predicted SBT:")
    print(rounded_predictions)

    # Save the rounded predicted SBT values to a text file
    new_data[6] = rounded_predictions
    new_data.to_csv('predicted_sbt.txt', sep='\t', index=False, header=False)

    # Extract the depth column from the new data
    depth = new_data.iloc[:, 0].values

    # Plot the predicted SBT vs depth
    plt.figure(figsize=(12, 90))
    plt.scatter(predictions, depth, label='Predicted SBT', color='blue', marker='o')
    plt.plot(rounded_predictions, depth, label='Rounded Predicted SBT', color='red', linestyle='-', linewidth=2)
    plt.xlabel('SBT')
    plt.ylabel('Depth')
    plt.title('Predicted SBT vs Depth')
    plt.xticks(np.arange(np.floor(np.min(predictions)), np.ceil(np.max(predictions))+1, 1))
    plt.yticks(np.arange(np.min(depth), np.max(depth)+0.02, 0.02))
    plt.legend()
    plt.grid(True)
    plt.show()


# Create a button for predicting with the trained model
predict_button = widgets.Button(description='Predict')
predict_button.on_click(predict_model)

# Display the 'Predict' button
display(predict_button)


IntSlider(value=2, description='Num Layers', max=5, min=1)

IntSlider(value=64, description='Num Neurons', max=256, min=16, step=16)

Button(description='Train', style=ButtonStyle())

Button(description='Predict', style=ButtonStyle())